In [25]:
import numpy as np
from numba import cuda
import time
import math # Import math for sqrt
import os

@cuda.jit
def potential_energy_kernel(masses, positions, potential_energies):
    """
    Numba CUDA 核心，用於計算每個粒子的重力位能。
    每個執行緒計算一個粒子 (i) 的位能貢獻。
    """
    i = cuda.grid(1)
    if i >= positions.shape[0]:
        return

    # G = 6.67430e-11  # 萬有引力常數
    G = 1.0
    
    local_energy = 0.0
    m_i = masses[i]
    pos_i = positions[i]

    # 計算粒子 i 與所有粒子 j (j > i) 之間的位能
    for j in range(i + 1, positions.shape[0]):
        m_j = masses[j]
        pos_j = positions[j]
        
        dx = pos_i[0] - pos_j[0]
        dy = pos_i[1] - pos_j[1]
        dz = pos_i[2] - pos_j[2]
        
        # 避免計算與自身的距離 (雖然迴圈已避免，但這是個好習慣)
        # 並加上一個很小的數 (epsilon) 來避免 r_sq == 0 的情況
        r_sq = dx*dx + dy*dy + dz*dz
        if r_sq < 1e-12: # epsilon^2
            r_sq = 1e-12
        inv_r = 1.0 / math.sqrt(r_sq)
        local_energy -= G * m_i * m_j * inv_r

    # 使用原子操作將局部能量加到總能量中
    cuda.atomic.add(potential_energies, 0, local_energy)

@cuda.jit
def kinetic_energy_kernel(masses, velocities, kinetic_energies):
    """
    Numba CUDA 核心，用於計算每個粒子的動能。
    每個執行緒計算一個粒子 (i) 的動能貢獻。
    """
    i = cuda.grid(1)
    if i >= velocities.shape[0]:
        return
    
    m_i = masses[i]
    vec_i = velocities[i]

    local_energy = 0.5 * m_i * (vec_i[0] * vec_i[0]
                              + vec_i[1] * vec_i[1]
                              + vec_i[2] * vec_i[2])

    # 使用原子操作將局部能量加到總能量中
    cuda.atomic.add(kinetic_energies, 0, local_energy)

@cuda.jit
def momentum_x_kernel(masses, velocities, momentum):
    """
    Numba CUDA 核心，用於計算每個粒子的動能。
    每個執行緒計算一個粒子 (i) 的動能貢獻。
    """
    i = cuda.grid(1)
    if i >= velocities.shape[0]:
        return

    m_i = masses[i]
    vec_i = velocities[i]

    local_momentum = m_i * vec_i[0]

    # 使用原子操作將局部能量加到總能量中
    cuda.atomic.add(momentum, 0, local_momentum)

@cuda.jit
def momentum_y_kernel(masses, velocities, momentum):
    """
    Numba CUDA 核心，用於計算每個粒子的動能。
    每個執行緒計算一個粒子 (i) 的動能貢獻。
    """
    i = cuda.grid(1)
    if i >= velocities.shape[0]:
        return

    m_i = masses[i]
    vec_i = velocities[i]

    local_momentum = m_i * vec_i[1]

    # 使用原子操作將局部能量加到總能量中
    cuda.atomic.add(momentum, 0, local_momentum)

@cuda.jit
def momentum_z_kernel(masses, velocities, momentum):
    """
    Numba CUDA 核心，用於計算每個粒子的動能。
    每個執行緒計算一個粒子 (i) 的動能貢獻。
    """
    i = cuda.grid(1)
    if i >= velocities.shape[0]:
        return

    m_i = masses[i]
    vec_i = velocities[i]

    local_momentum = m_i * vec_i[2]

    # 使用原子操作將局部能量加到總能量中
    cuda.atomic.add(momentum, 0, local_momentum)

def calculate_potential_energy_gpu(filename="uniform_sphere_3d_1e6.bin", n_particles=1_000_000):
    """
    使用 Numba 在 GPU 上計算總重力位能。
    """
    # 每個粒子的數據結構：[mass, x, y, z, vx, vy, vz] (7個 float64)
    
    # 讀取二進位檔案

    def load_particles_bin(filename, n_particles):
        with open(filename, "rb") as f:
            data = np.fromfile(f, dtype=np.float64).reshape(n_particles, 7)
        return data
    particles = load_particles_bin(filename, n_particles)

    num_particles = len(particles)
    print(f"成功讀取 {num_particles} 個粒子。")

    # 提取質量和位置
    masses = particles[:, 0]
    masses = np.ascontiguousarray(masses)
    positions = np.ascontiguousarray(np.vstack((particles[:, 1], particles[:, 2], particles[:, 3])).T.copy())
    velocities = np.ascontiguousarray(np.vstack((particles[:, 4], particles[:, 5], particles[:, 6])).T.copy())

    start_time = time.time()

    # 將數據傳輸到 GPU
    d_masses = cuda.to_device(masses)
    d_positions = cuda.to_device(positions)
    d_velocities = cuda.to_device(velocities)
    d_potential_energies = cuda.to_device(np.zeros(1, dtype=np.float64))
    d_kinetic_energies = cuda.to_device(np.zeros(1, dtype=np.float64))
    d_momentum_x = cuda.to_device(np.zeros(1, dtype=np.float64))
    d_momentum_y = cuda.to_device(np.zeros(1, dtype=np.float64))
    d_momentum_z = cuda.to_device(np.zeros(1, dtype=np.float64))

    # 設定 CUDA 核心的執行配置
    threads_per_block = 256
    blocks_per_grid = (num_particles + (threads_per_block - 1)) // threads_per_block

    # 執行 CUDA 核心
    potential_energy_kernel[blocks_per_grid, threads_per_block](
        d_masses, d_positions, d_potential_energies
    )
    kinetic_energy_kernel[blocks_per_grid, threads_per_block](
        d_masses, d_velocities, d_kinetic_energies
    )
    momentum_x_kernel[blocks_per_grid, threads_per_block](
        d_masses, d_velocities, d_momentum_x
    )
    momentum_y_kernel[blocks_per_grid, threads_per_block](
        d_masses, d_velocities, d_momentum_y
    )
    momentum_z_kernel[blocks_per_grid, threads_per_block](
        d_masses, d_velocities, d_momentum_z
    )
    
    # 將結果從 GPU 傳回 CPU
    total_potential_energy = d_potential_energies.copy_to_host()
    total_kinetic_energy = d_kinetic_energies.copy_to_host()
    total_momentum_x = d_momentum_x.copy_to_host()
    total_momentum_y = d_momentum_y.copy_to_host()
    total_momentum_z = d_momentum_z.copy_to_host()

    end_time = time.time()

    print(f"計算完成。")
    print(f"總重力位能: {total_potential_energy[0]:.6e} J")
    print(f"總動能: {total_kinetic_energy[0]:.6e} J")
    print(f"總動量: {total_momentum_x[0]:.6e} {total_momentum_y[0]:.6e} {total_momentum_z[0]:.6e}")
    print(f"GPU 計算耗時: {end_time - start_time:.4f} 秒")

if __name__ == "__main__":
    calculate_potential_energy_gpu()

成功讀取 1000000 個粒子。
計算完成。
總重力位能: -1.200422e+10 J
總動能: 2.501326e+05 J
總動量: -6.263654e+01 7.146460e+02 0.000000e+00
GPU 計算耗時: 44.0081 秒


In [ ]:
import numpy as np
import pandas as pd

def run_integration_step(dt=0.01, filename='plummer_vel_3d_1e5.bin'):
    """
    執行一步 KDK 軌道積分。
    """
    # --- 讀取資料 ---
    print("\n--- 開始 KDK 積分步驟 ---")

    # 1. 讀取粒子初始狀態 (mass, pos, vel)
    # 使用 np.fromfile 高效讀取二進位檔
    try:
        particle_data = np.fromfile(filename, dtype=np.float64)
        # 將一維陣列重塑為 (N, 7) 的形狀
        particles = particle_data.reshape(-1, 7)
    except FileNotFoundError:
        print("錯誤: particles.bin 不存在。請先執行資料準備腳本。")
        return

    mass = particles[:, 0:1]  # 維持 (N, 1) 形狀以便廣播
    pos = particles[:, 1:4]   # 位置 (x, y, z)
    vel = particles[:, 4:7]   # 速度 (vx, vy, vz)
    print(f"成功讀取 {len(particles)} 顆粒子。")

    # 2. 讀取 tree2fmm 計算出的力，並計算加速度 a = F/m
    try:
        force_df = pd.read_csv('force_tree2fmm.csv', header=None)
        force = force_df.to_numpy()
        # 核心計算：加速度 a = F/m。NumPy的廣播機制會自動處理維度。
        accel = force / mass
    except FileNotFoundError:
        print("錯誤: force_tree2fmm.csv 不存在。請確保 tree2fmm 已成功執行。")
        return
    print("成功讀取力，並計算出加速度。")


    # --- KDK (Kick-Drift-Kick) 演算法 ---
    # KDK 分為三步: Kick (半步) -> Drift (全步) -> Kick (半步)
    # v(t + dt/2) = v(t) + a(t) * dt/2
    # x(t + dt)   = x(t) + v(t + dt/2) * dt
    # v(t + dt)   = v(t + dt/2) + a(t + dt) * dt/2

    # 3. Kick (第一步): 使用目前的加速度 a(t) 更新速度半個時間步
    vel_half = vel + accel * (dt / 2.0)

    # 4. Drift: 使用更新後的速度 vel_half 更新位置一整個時間步
    pos_new = pos + vel_half * dt

    # 5. 更新粒子狀態並儲存，為下一次計算力做準備
    # 注意：KDK的最後一步 (第二次Kick) 需要下一個時間點的加速度 a(t+dt)
    # 所以完整的模擬循環是：
    #   a. 計算 a(t) (透過 tree2fmm)
    #   b. 更新速度半步、位置一步 (如上)
    #   c. 用新的位置 pos_new 再次執行 tree2fmm 得到 a(t+dt)
    #   d. 用 a(t+dt) 完成速度的後半步更新
    #
    # 為了簡化這個 Demo，我們只做到 Drift 結束，並儲存新的位置。
    
    print("\n--- 積分計算完成 ---")
    print(f"時間步長 (dt): {dt}")
    print("\n前3顆粒子的位置變化:")
    for i in range(3):
        print(f"  粒子 {i}:")
        print(f"    舊位置: {pos[i]}")
        print(f"    新位置: {pos_new[i]}")

    # 6. 將更新後的粒子資料寫回檔案，以便進行下一個循環
    #   (這個範例中，我們只更新了位置，速度尚未完成第二次 kick)
    #   一個完整的程式需要將 vel_half 和 pos_new 傳遞給下個階段
    updated_particles = np.hstack((mass, pos_new, vel_half)) # 暫存 vel_half
    # 可以將 updated_particles 寫入新的 .bin 檔案，作為下一次 tree2fmm 的輸入
    updated_particles.tofile('particles_new.bin')
    print("\n新的粒子位置已儲存至 particles_new.bin")
    print("下一步：請使用 a_new.bin 執行 tree2fmm 以獲得新的力。")


if __name__ == '__main__':
    # 執行一次積分步驟，時間步長為 0.01
    run_integration_step(dt=0.01)


--- 開始 KDK 積分步驟 ---
成功讀取 100000 顆粒子。
成功讀取力，並計算出加速度。

--- 積分計算完成 ---
時間步長 (dt): 0.01

前3顆粒子的位置變化:
  粒子 0:
    舊位置: [ 13.53010012  -8.98560958 -35.19231685]
    新位置: [ 13.62258889  -9.048065   -35.08313975]
  粒子 1:
    舊位置: [-15.6188977   54.06134491  -2.6246147 ]
    新位置: [-15.82124861  53.89944727  -2.53453352]
  粒子 2:
    舊位置: [-45.06762603   2.60466456 -10.62268438]
    新位置: [-44.88025915   2.7455819  -10.48275211]

新的粒子位置已儲存至 particles_new.bin
下一步：請使用 a_new.bin 執行 tree2fmm 以獲得新的力。
